In [1]:
from glob import glob

import pandas as pd

#최종 df 먼저 생성 
df = []
years = [2021,2022,2023]

for year in years:
    datas = glob(f'./datas/{year}/*.xlsx')

    teams = [xlsx.split('_')[-2] for xlsx in datas]

    # concat 기준이 될 df 먼저 생성, 
    df_year = pd.read_excel(f'{datas[0]}', engine='openpyxl')

    # H/A 헷갈려서 기준 잡는 열 생성 / 시즌 열 생성
    df_year['기준'] = teams[0]
    df_year['시즌'] = year

    # 새로운 df 불러오고 기준 df에 concat하기
    for xlsx in datas[1:]:
        team_name = xlsx.split('_')[-2]
        df_temp = pd.read_excel(f'{xlsx}', engine="openpyxl")
        df_temp["기준"] = f'{team_name}'
        df_temp["시즌"] = year
        df_year = pd.concat([df_year, df_temp])
    df.append(df_year)

df = pd.concat(df, ignore_index=True)


# K_Columns 에서 컬럼명 가져오기
names = pd.read_excel('datas/K_columns.xlsx', engine="openpyxl")


# 필요없는 열 제거 (성공%만 가져옴)
temp = names.drop(columns=['LABEL-45','LABEL-46','LABEL-47','LABEL-48']) 


# 컬럼명 가져와서 리스트로 저장
rows = temp.iloc[0].to_list()


# df의 컬럼 개수를 K_columns에 맞춰 개수 맞추기
drop_columns = ['14', '15','20','21','23','24','27', '28' ,'30' ,'31' ,'33' ,'34', '36', '37', '39', '40','42', '43' ,'45',
               '46' ,'48', '49' ,'51' ,'52' ,'54', '55', '58', '59', '61', '62', '64', '65']
for idx in (drop_columns):
    df = df.drop(columns=[f'LABEL-{idx}'])


# 컬럼명 변경 ("기준"열 제외한)
for idx, column in enumerate(df.columns[:-2]):
    df = df.rename(columns={f"{column}": f"{rows[idx]}"})


# 대회별 득점 비교 후 경기결과 열에 저장
for i in df["대회"].value_counts().index.tolist():
    for j in range(1,39): # 총 35개 라운드 수집
        H_score = df[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME")]["득점"]
        A_score = df[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY")]["득점"]
        if len(H_score) > 0:
            if H_score.iloc[0] > A_score.iloc[0]:
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME"), "경기결과"] = 'W'
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY"), "경기결과"] = 'L'
            elif H_score.iloc[0] < A_score.iloc[0]:
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME"), "경기결과"] = 'L'
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY"), "경기결과"] = 'W'
            else:
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "HOME"), "경기결과"] = 'D'
                df.loc[(df["대회"] == i) & (df["Rnd."] == j) & (df["H/A"] == "AWAY"), "경기결과"] = 'D'


# 컬럼명 변경 확인
df.head()

,Rnd.,대회,H/A,득점,도움,슈팅,유효 슈팅,블락된슈팅,벗어난슈팅,PA내 슈팅,...,획득,블락,볼미스,파울,피파울,경고,퇴장,기준,시즌,경기결과
0,1,대구vs수원FC,HOME,1,1,13,4,2,7,5,...,87,3,9,13,14,1,0,daegu,2021,D
1,2,인천vs대구,AWAY,1,0,14,2,5,7,7,...,102,2,10,13,22,1,0,daegu,2021,L
2,3,대구vs광주,HOME,1,0,17,4,6,7,10,...,96,2,12,18,14,2,0,daegu,2021,L
3,4,대구vs제주,HOME,1,0,16,4,3,9,10,...,102,2,14,13,14,2,0,daegu,2021,D
4,5,전북vs대구,AWAY,2,2,17,4,10,3,10,...,102,3,5,13,15,4,1,daegu,2021,L


In [2]:
# transfer markt 데이터 불러오고 불필요 열 제거
tm_data_2021 = pd.read_excel('datas/K_trsfmk_2021.xlsx', engine='openpyxl').drop(columns=['LABEL-1','LABEL-2','LABEL-4','LABEL-5'])
tm_data_2022 = pd.read_excel('datas/K_trsfmk_2022.xlsx', engine='openpyxl').drop(columns=['LABEL-1','LABEL-2','LABEL-4'])
tm_data_2023 = pd.read_excel('datas/K_trsfmk_2023.xlsx', engine='openpyxl').drop(columns=['LABEL-1','LABEL-2','LABEL-4'])

# 열 이름 변경
tm_data_2021 = tm_data_2021.rename(columns={'LABEL-3' : '구단', 'LABEL-6' : '소속 선수 수',
                                'LABEL-7' : '평균 나이','LABEL-8' : '용병 수',
                                'LABEL-9' : '평균 가치', 'LABEL-10' : '총 가치'})

tm_data_2022 = tm_data_2022.rename(columns={'LABEL-3' : '구단', 'LABEL-5' : '소속 선수 수',
                                'LABEL-6' : '평균 나이','LABEL-7' : '용병 수',
                                'LABEL-8' : '평균 가치', 'LABEL-9' : '총 가치'})

tm_data_2023 = tm_data_2023.rename(columns={'LABEL-3' : '구단', 'LABEL-5' : '소속 선수 수',
                                'LABEL-6' : '평균 나이','LABEL-7' : '용병 수',
                                'LABEL-8' : '평균 가치', 'LABEL-9' : '총 가치'})


# 구단 이름 수정
tm_data_2021['구단'] = ['jeonbuk', 'ulsan', 'seoul', 'suwon', 'incheon', 'suwonFC',
                           'gangwon', 'jeju', 'pohang', 'seongnam','daegu','gwangju']

tm_data_2022['구단'] = ['jeonbuk', 'seoul', 'gimcheon', 'ulsan', 'suwon', 'jeju',
                           'pohang', 'incheon', 'gangwon', 'seongnam','daegu','suwonFC']

tm_data_2023['구단'] = ['jeonbuk', 'ulsan', 'seoul', 'suwon', 'daejeon', 'gangwon',
                           'jeju', 'incheon', 'gwangju', 'pohang','daegu','suwonFC']


# 평균 가치, 총 가치 열에서 €, k, m 제거
tm_data_2021['평균 가치'] = tm_data_2021['평균 가치'].str.replace('€', '').str.replace('k', '').astype(float)
tm_data_2021['총 가치'] = tm_data_2021['총 가치'].str.replace('€', '').str.replace('m', '').astype(float)

tm_data_2022['평균 가치'] = tm_data_2022['평균 가치'].str.replace('€', '').str.replace('k', '').astype(float)
tm_data_2022['총 가치'] = tm_data_2022['총 가치'].str.replace('€', '').str.replace('m', '').astype(float)

tm_data_2023['평균 가치'] = tm_data_2023['평균 가치'].str.replace('€', '').str.replace('k', '').astype(float)
tm_data_2023['총 가치'] = tm_data_2023['총 가치'].str.replace('€', '').str.replace('m', '').astype(float)


# df에 tm_data_2023 합치기
tm_columns = tm_data_2021.columns[1:]

# df에 열 먼저 생성
for column in tm_columns:
    df[f'{column}'] = None
# df.head(1)

# 2021
for column in tm_data_2021.columns[1:]:
    for i in df['기준']:
        for j in tm_data_2021['구단']:
            if i == j:
                df.loc[(df['기준'] == i) &(df['시즌'] == 2021), f'{column}'] = tm_data_2021.loc[tm_data_2021['구단'] == j , f'{column}'].values[0]
print('2021 end !')

#2022
for column in tm_data_2022.columns[1:]:
    for i in df['기준']:
        for j in tm_data_2022['구단']:
            if i == j:
                df.loc[(df['기준'] == i) &(df['시즌'] == 2022), f'{column}'] = tm_data_2022.loc[tm_data_2022['구단'] == j , f'{column}'].values[0]
print('2022 end !')

#2023
for column in tm_data_2023.columns[1:]:
    for i in df['기준']:
        for j in tm_data_2023['구단']:
            if i == j:
                df.loc[(df['기준'] == i) &(df['시즌'] == 2023), f'{column}'] = tm_data_2023.loc[tm_data_2023['구단'] == j , f'{column}'].values[0]
print('2023 end !')

2021 end !
2022 end !
2023 end !


In [3]:
# df.to_csv('K-League-data22.csv')

In [4]:
# 프레임 셰입 변경

new_rows = []

unique_competitions = df['대회'].unique()

for competition in unique_competitions:
    competition_rows = df[df['대회'] == competition]
    
    unique_seasons = competition_rows['시즌'].unique()
    
    for season in unique_seasons:
        season_rows = competition_rows[competition_rows['시즌'] == season]
        
        unique_rounds = season_rows['Rnd.'].unique()
        
        for rnd in unique_rounds:
            round_rows = season_rows[season_rows['Rnd.'] == rnd]
            
            home_row = round_rows[round_rows['H/A'] == 'HOME'].iloc[0]
            away_row = round_rows[round_rows['H/A'] == 'AWAY'].iloc[0]
            
            merged_row = pd.concat([home_row, away_row]).reset_index(drop=True)
            new_rows.append(merged_row)

new_df = pd.DataFrame(new_rows)

# print(new_df)


merged_columns = ['Rnd.', '대회', 'H/A', '홈 득점', '홈 도움', '홈 슈팅', '홈 유효 슈팅', '홈 블락된슈팅', '홈 벗어난슈팅',
       '홈 PA내 슈팅', '홈 PA외 슈팅', '홈 프리킥 슈팅', '홈 프리킥 유효슈팅', '홈 프리킥 크로스', '홈 오프사이드', '홈 코너킥',
       '홈 스로인', '홈 드리블', '홈 패스', '홈 키패스', '홈 전방패스', '홈 후방패스', '홈 횡패스', '홈 공격진영 패스', '홈 수비진영 패스',
       '홈 중앙진영 패스', '홈 롱패스', '홈 중거리패스', '홈 단거리패스', '홈 크로스', '홈 탈압박', '홈 경합 지상', '홈 경합 공중',
       '홈 태클', '홈 클리어링', '홈 인터셉트', '홈 차단', '홈 획득', '홈 블락', '홈 볼미스', '홈 파울', '홈 피파울', '홈 경고', '홈 퇴장',
       '기준', '시즌', '홈 경기결과', '홈 소속 선수 수', '홈 평균 나이', '홈 용병 수', '홈 평균 가치', '홈 총 가치',
       
       'Rnd.2', '대회2', 'H/A2', '원정 득점', '원정 도움', '원정 슈팅', '원정 유효 슈팅', '원정 블락된슈팅', '원정 벗어난슈팅',
       '원정 PA내 슈팅', '원정 PA외 슈팅', '원정 프리킥 슈팅', '원정 프리킥 유효슈팅', '원정 프리킥 크로스', '원정 오프사이드', '원정 코너킥',
       '원정 스로인', '원정 드리블', '원정 패스', '원정 키패스', '원정 전방패스', '원정 후방패스', '원정 횡패스', '원정 공격진영 패스', '원정 수비진영 패스',
       '원정 중앙진영 패스', '원정 롱패스', '원정 중거리패스', '원정 단거리패스', '원정 크로스', '원정 탈압박', '원정 경합 지상', '원정 경합 공중',
       '원정 태클', '원정 클리어링', '원정 인터셉트', '원정 차단', '원정 획득', '원정 블락', '원정 볼미스', '원정 파울', '원정 피파울', '원정 경고', '원정 퇴장',
       '기준2', '시즌2', '원정 경기결과', '원정 소속 선수 수', '원정 평균 나이', '원정 용병 수', '원정 평균 가치', '원정 총 가치']

new_df.columns = merged_columns

# 불필요 칼럼 지우기
new_df = new_df.drop(columns=['Rnd.2', '대회2', '시즌2', 'H/A', 'H/A2'])
new_df = new_df.rename(columns={'기준':'홈 팀', '기준2':'원정 팀'})

In [5]:
new_df.to_csv('K-League-data.csv')

In [6]:
print(len(new_df.columns))

99


In [11]:
for i in new_df.columns:
    print(i)

Rnd.
대회
홈 득점
홈 도움
홈 슈팅
홈 유효 슈팅
홈 블락된슈팅
홈 벗어난슈팅
홈 PA내 슈팅
홈 PA외 슈팅
홈 프리킥 슈팅
홈 프리킥 유효슈팅
홈 프리킥 크로스
홈 오프사이드
홈 코너킥
홈 스로인
홈 드리블
홈 패스
홈 키패스
홈 전방패스
홈 후방패스
홈 횡패스
홈 공격진영 패스
홈 수비진영 패스
홈 중앙진영 패스
홈 롱패스
홈 중거리패스
홈 단거리패스
홈 크로스
홈 탈압박
홈 경합 지상
홈 경합 공중
홈 태클
홈 클리어링
홈 인터셉트
홈 차단
홈 획득
홈 블락
홈 볼미스
홈 파울
홈 피파울
홈 경고
홈 퇴장
홈 팀
시즌
홈 경기결과
홈 소속 선수 수
홈 평균 나이
홈 용병 수
홈 평균 가치
홈 총 가치
원정 득점
원정 도움
원정 슈팅
원정 유효 슈팅
원정 블락된슈팅
원정 벗어난슈팅
원정 PA내 슈팅
원정 PA외 슈팅
원정 프리킥 슈팅
원정 프리킥 유효슈팅
원정 프리킥 크로스
원정 오프사이드
원정 코너킥
원정 스로인
원정 드리블
원정 패스
원정 키패스
원정 전방패스
원정 후방패스
원정 횡패스
원정 공격진영 패스
원정 수비진영 패스
원정 중앙진영 패스
원정 롱패스
원정 중거리패스
원정 단거리패스
원정 크로스
원정 탈압박
원정 경합 지상
원정 경합 공중
원정 태클
원정 클리어링
원정 인터셉트
원정 차단
원정 획득
원정 블락
원정 볼미스
원정 파울
원정 피파울
원정 경고
원정 퇴장
원정 팀
원정 경기결과
원정 소속 선수 수
원정 평균 나이
원정 용병 수
원정 평균 가치
원정 총 가치


In [7]:
'Rnd.', '대회', '홈 득점', '홈 도움', '홈 슈팅', '홈 유효 슈팅', '홈 블락된슈팅', '홈 벗어난슈팅',
'홈 PA내 슈팅', '홈 PA외 슈팅', '홈 프리킥 슈팅', '홈 프리킥 유효슈팅', '홈 프리킥 크로스',
'홈 오프사이드', '홈 코너킥', '홈 스로인', '홈 드리블', '홈 패스', '홈 키패스', '홈 전방패스',
'홈 후방패스', '홈 횡패스', '홈 공격진영 패스', '홈 수비진영 패스', '홈 중앙진영 패스', '홈 롱패스',
'홈 중거리패스', '홈 단거리패스', '홈 크로스', '홈 탈압박', '홈 경합 지상', '홈 경합 공중', '홈 태클',
'홈 클리어링', '홈 인터셉트', '홈 차단', '홈 획득', '홈 블락', '홈 볼미스', '홈 파울', '홈 피파울',
'홈 경고', '홈 퇴장', '홈 팀', '시즌', '홈 경기결과', '홈 소속 선수 수', '홈 평균 나이', '홈 용병 수',
'홈 평균 가치', '홈 총 가치', '원정 득점', '원정 도움', '원정 슈팅', '원정 유효 슈팅', '원정 블락된슈팅',
'원정 벗어난슈팅', '원정 PA내 슈팅', '원정 PA외 슈팅', '원정 프리킥 슈팅', '원정 프리킥 유효슈팅',
'원정 프리킥 크로스', '원정 오프사이드', '원정 코너킥', '원정 스로인', '원정 드리블', '원정 패스',
'원정 키패스', '원정 전방패스', '원정 후방패스', '원정 횡패스', '원정 공격진영 패스', '원정 수비진영 패스',
'원정 중앙진영 패스', '원정 롱패스', '원정 중거리패스', '원정 단거리패스', '원정 크로스', '원정 탈압박',
'원정 경합 지상', '원정 경합 공중', '원정 태클', '원정 클리어링', '원정 인터셉트', '원정 차단', '원정 획득',
'원정 블락', '원정 볼미스', '원정 파울', '원정 피파울', '원정 경고', '원정 퇴장', '원정 팀',
'원정 경기결과', '원정 소속 선수 수', '원정 평균 나이', '원정 용병 수', '원정 평균 가치', '원정 총 가치'

('원정 경기결과', '원정 소속 선수 수', '원정 평균 나이', '원정 용병 수', '원정 평균 가치', '원정 총 가치')

In [8]:
new_df['대회'].value_counts()

대회
울산vs전북      7
전북vs포항      7
대구vs수원FC    6
포항vs울산      6
울산vs제주      6
           ..
전북vs김천      1
대구vs대전      1
제주vs김천      1
포항vs김천      1
대전vs전북      1
Name: count, Length: 176, dtype: int64

In [9]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # 예제 데이터 생성
# data = {
#     '홈 팀': ['팀A', '팀B', '팀C', '팀A', '팀B', '팀C', '팀A', '팀B'],
#     '원정 팀': ['팀B', '팀C', '팀A', '팀C', '팀A', '팀B', '팀C', '팀A'],
#     '홈 득점': [1, 2, 1, 2, 3, 1, 2, 1],
#     '원정 득점': [0, 1, 2, 1, 1, 2, 1, 2],
#     '홈 슈팅': [10, 8, 7, 9, 12, 6, 10, 9],
#     '원정 슈팅': [5, 7, 8, 6, 8, 7, 6, 8],
#     '결과': [1, 0, 1, 0, 1, 0, 1, 0]  # 홈팀 승리(1), 원정팀 승리(0)
# }

# df = pd.DataFrame(data)

# # 팀별 평균 통계 계산
# numeric_columns = ['홈 득점', '원정 득점', '홈 슈팅', '원정 슈팅']
# home_team_stats = df.groupby('홈 팀')[numeric_columns].mean().add_prefix('홈_')
# away_team_stats = df.groupby('원정 팀')[numeric_columns].mean().add_prefix('원정_')

# # 홈 팀과 원정 팀 통계를 하나의 데이터프레임으로 병합
# team_stats = home_team_stats.join(away_team_stats, how='outer')
# print(team_stats)

# # 특징과 레이블 선택
# features = df.drop(columns=['결과'])
# labels = df['결과']

# # 데이터 전처리
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
# scaler = StandardScaler()

# # 팀 통계로 훈련 데이터 확장
# def get_team_stats(row, stats):
#     home_team = row['홈 팀']
#     away_team = row['원정 팀']
#     home_stats = stats.loc[home_team].values if home_team in stats.index else [0] * len(numeric_columns)
#     away_stats = stats.loc[away_team].values if away_team in stats.index else [0] * len(numeric_columns)
#     return pd.Series(list(home_stats) + list(away_stats))

# X_train_stats = X_train.apply(get_team_stats, axis=1, stats=team_stats)
# X_test_stats = X_test.apply(get_team_stats, axis=1, stats=team_stats)

# X_train_scaled = scaler.fit_transform(X_train_stats)
# X_test_scaled = scaler.transform(X_test_stats)

# # 모델 학습
# model = RandomForestClassifier(random_state=42)
# model.fit(X_train_scaled, y_train)

# # 예측
# y_pred = model.predict(X_test_scaled)

# # 평가
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')

# # 새로운 데이터로 예측
# new_game = {'홈 팀': '팀A', '원정 팀': '팀B'}
# new_features = get_team_stats(new_game, team_stats).values.reshape(1, -1)
# new_features_scaled = scaler.transform(new_features)
# prediction = model.predict(new_features_scaled)
# print(f'Prediction: {prediction[0]}')  # 예측 결과 출력